In [ ]:
# https://docs.google.com/document/d/1lrYEnBW5aqioXriEt47KI_o3JUsCK9fShqySyNbKUn0/edit

In [1]:
import numpy as np
import pandas as pd
import pprint as pprint

In [2]:
train_data = pd.read_csv("sign_mnist_train.csv")
y_train = train_data['label'].to_numpy()
x_train = train_data.drop(['label'], axis = 1).to_numpy()

train_data.shape

(27455, 785)

In [3]:
test_data = pd.read_csv("sign_mnist_test.csv")
y_test = test_data['label'].to_numpy()
x_test = test_data.drop(['label'], axis = 1).to_numpy()

test_data.shape

(7172, 785)

In [4]:
classes = train_data['label'].unique()
classes

array([ 3,  6,  2, 13, 16,  8, 22, 18, 10, 20, 17, 19, 21, 23, 24,  1, 12,
       11, 15,  4,  0,  5,  7, 14], dtype=int64)

In [5]:
num_classes = len(classes)
num_classes

24

In [6]:
x_train = np.array([np.reshape(i, (28,28)) for i in x_train])
x_test = np.array([np.reshape(i, (28,28)) for i in x_test])

In [7]:
x_train.shape

(27455, 28, 28)

In [8]:
# x_train = np.array(x_train.iloc[:,:])
x_train = np.array([np.reshape(i, (28,28)) for i in x_train])
# x_test = np.array(x_test.iloc[:,:])
x_test = np.array([np.reshape(i, (28,28)) for i in x_test])
num_classes = 26
y_train = np.array(y_train).reshape(-1)
y_test = np.array(y_test).reshape(-1)
y_train = np.eye(num_classes)[y_train]
y_test = np.eye(num_classes)[y_test]
x_train = x_train.reshape((27455, 28, 28, 1))
x_test = x_test.reshape((7172, 28, 28, 1))

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [10]:
classifier = keras.Sequential()
classifier.add(layers.Conv2D(filters=8, kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(28,28,1),activation='relu', data_format='channels_last'))
classifier.add(layers.MaxPooling2D(pool_size=(2,2)))
classifier.add(layers.Conv2D(filters=16, kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
classifier.add(layers.Dropout(0.5))
classifier.add(layers.MaxPooling2D(pool_size=(4,4)))
classifier.add(layers.Dense(128, activation='relu'))
classifier.add(layers.Flatten())
classifier.add(layers.Dense(26, activation='softmax'))

In [11]:
classifier.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(x_train, y_train, epochs=50, batch_size=100)

Epoch 1/50
275/275 [==============================] - 28s 86ms/step - loss: 3.5898 - accuracy: 0.0461
Epoch 2/50
275/275 [==============================] - 15s 55ms/step - loss: 3.1684 - accuracy: 0.0644
Epoch 3/50
275/275 [==============================] - 14s 50ms/step - loss: 3.0139 - accuracy: 0.1103
Epoch 4/50
275/275 [==============================] - 14s 51ms/step - loss: 2.1941 - accuracy: 0.3413
Epoch 5/50
275/275 [==============================] - 13s 46ms/step - loss: 1.2776 - accuracy: 0.5993
Epoch 6/50
275/275 [==============================] - 13s 45ms/step - loss: 0.8933 - accuracy: 0.7155
Epoch 7/50
275/275 [==============================] - 14s 50ms/step - loss: 0.7253 - accuracy: 0.7660
Epoch 8/50
275/275 [==============================] - 14s 52ms/step - loss: 0.6239 - accuracy: 0.7908
Epoch 9/50
275/275 [==============================] - 14s 51ms/step - loss: 0.5568 - accuracy: 0.8152
Epoch 10/50
275/275 [==============================] - 14s 49ms/step - loss: 0.530

In [12]:
accuracy = classifier.evaluate(x = x_test, y = y_test, batch_size=32)
print("Accuracy: ",accuracy[1])

225/225 [==============================] - 3s 12ms/step - loss: 0.5164 - accuracy: 0.8275
Accuracy:  0.8275237083435059


In [58]:
classifier.save('CNNmodel.h5')
weights_file = drive.CreateFile({'title' : 'CNNmodel.h5'})
weights_file.SetContentFile('CNNmodel.h5')
weights_file.Upload()
drive.CreateFile({'id': weights_file.get('id')})

NameError: name 'drive' is not defined

In [61]:
import cv2
import os
import time
import argparse
import cv2
import numpy as np
from keras.models import load_model
from skimage.transform import resize, pyramid_reduce
import PIL
from PIL import Image

In [60]:
def keras_predict(model, image):
    data = np.asarray( image, dtype="int32" )
    pred_probab = model.predict(data)[0]
    
    # softmax gives probability for all the alphabets hence we have to choose the maximum probability alphabet 
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class

In [1]:
vid = cv2.VideoCapture(0)
img_counter = 0

while(True):
    
    # ret, frame = vid.read()
    ret, frame = vid.read()
    cv2.imshow('frame', frame)
      
    # the 'q' button : quitting button 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # capturing the image from webcam 
        image_frame = vid.read()
        
        # # to crop required part
        # im2 = crop_image(image_frame, 300,300,300,300)
        im2 = image_frame

        # convert to grayscale 
        image_grayscale = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

        # blurring the image 
        image_grayscale_blurred =cv2.GaussianBlur(image_grayscale, (15,15), 0)
        
        # resize the image to 28x28
        im3 = cv2.resize(image_grayscale_blurred, (28,28), interpolation = cv2.INTER_AREA)

        # expand the dimensions from 28x28 to 1x28x28x1
        im4 = np.resize(im3, (28, 28, 1))
        im5 = np.expand_dims(im4, axis=0)
        
        pred_prob, pred_class = keras_predict(classifier, im5)
        print(pred_class)
        print(pred_prob)
        time.sleep(15)
        break

# After the loop release the cap object
vid.release()

# Destroy all the windows
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [ ]:
 # img_name = "opencv_frame_{}.png".format(0)
        # cv2.imwrite(img_name, frame)
        # print("{} written!".format(img_name))
        # img_counter += 1
        
#         time.sleep(10)

        
#         image = cv2.imread( "opencv_frame_{}.png".format(0))        
#         # Select ROI
#         # image = cv2.imread("11.jpg")
#         r = cv2.selectROI("select the area", image)

#         # Crop image
#         cropped_image = image[int(r[1]):int(r[1]+r[3]),
#                               int(r[0]):int(r[0]+r[2])]
        
#         img_name = "opencv_frame_{}.png".format(img_counter)
#         cv2.imwrite(img_name, cropped_image)
        
#         # Display cropped image
#         cv2.imshow("Cropped image", cropped_image)